In [51]:
import sys
sys.path.append('/tmp-data/zhx/DriverOrderOfflineRL/cage-challenge-1/CybORG')
sys.path.append('/tmp-data/zhx/DriverOrderOfflineRL/tianshou')
sys.path.append('/tmp-data/zhx/DriverOrderOfflineRL/gym')

### args define

In [52]:
import argparse
import datetime
import os
import pprint

import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch import nn

from tianshou.data import Collector, VectorReplayBuffer
from tianshou.policy import DQNPolicy
from tianshou.policy.modelbased.icm import ICMPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils import TensorboardLogger, WandbLogger
from tianshou.utils.net.discrete import IntrinsicCuriosityModule
from tianshou.env import SubprocVectorEnv, DummyVectorEnv

from typing import Any, Dict, Optional, Sequence, Tuple, Union


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--task", type=str, default="cyborg")
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--scale-obs", type=int, default=0)
    parser.add_argument("--eps-test", type=float, default=0.05)
    parser.add_argument("--eps-train", type=float, default=1.)
    parser.add_argument("--eps-train-final", type=float, default=0.05)
    parser.add_argument("--buffer-size", type=int, default=5000)
    parser.add_argument("--lr", type=float, default=0.0001)
    parser.add_argument("--gamma", type=float, default=0.99)
    parser.add_argument("--n-step", type=int, default=1)
    parser.add_argument("--target-update-freq", type=int, default=1000)
    parser.add_argument("--epoch", type=int, default=2000)
    parser.add_argument("--step-per-epoch", type=int, default=10)
    parser.add_argument("--step-per-collect", type=int, default=100)
    parser.add_argument("--update-per-step", type=float, default=1)
    parser.add_argument("--batch-size", type=int, default=32)
    parser.add_argument("--training-num", type=int, default=10)
    parser.add_argument("--test-num", type=int, default=10)
    parser.add_argument("--logdir", type=str, default="log")
    parser.add_argument("--render", type=float, default=0.)
    parser.add_argument(
        "--device", type=str, default="cuda" if torch.cuda.is_available() else "cpu"
    )
    parser.add_argument("--frames-stack", type=int, default=1)
    parser.add_argument("--resume-path", type=str, default=None)
    parser.add_argument("--resume-id", type=str, default=None)
    parser.add_argument(
        "--logger",
        type=str,
        default="wandb",
        choices=["tensorboard", "wandb"],
    )
    parser.add_argument("--wandb-project", type=str, default="cyborg.dqn")
    parser.add_argument(
        "--watch",
        default=False,
        action="store_true",
        help="watch the play of pre-trained policy only"
    )
    parser.add_argument("--save-buffer-name", type=str, default=None)
    parser.add_argument(
        "--icm-lr-scale",
        type=float,
        default=0.,
        help="use intrinsic curiosity module with this lr scale"
    )
    parser.add_argument(
        "--icm-reward-scale",
        type=float,
        default=0.01,
        help="scaling factor for intrinsic curiosity reward"
    )
    parser.add_argument(
        "--icm-forward-loss-weight",
        type=float,
        default=0.2,
        help="weight for the forward model loss in ICM"
    )
    return parser.parse_args(args=[])


### env

In [53]:
args = get_args()

In [54]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Shared.Actions import *
from CybORG.Agents import RedMeanderAgent, B_lineAgent
from CybORG.Agents.Wrappers import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

# seed
np.random.seed(args.seed)
torch.manual_seed(args.seed)

# env
CYBORG = CybORG(path,'sim', agents={'Red': RedMeanderAgent})
env = ChallengeWrapper(env=CYBORG, agent_name="Blue", max_steps=args.step_per_epoch)
train_envs = DummyVectorEnv([lambda: ChallengeWrapper(env=CybORG(path,'sim', agents={'Red': RedMeanderAgent}), agent_name="Blue", max_steps=args.step_per_epoch) for _ in range(1)])
test_envs = DummyVectorEnv([lambda: ChallengeWrapper(env=CybORG(path,'sim', agents={'Red': RedMeanderAgent}), agent_name="Blue", max_steps=args.step_per_epoch) for _ in range(1)])

In [55]:
args.state_shape = env.observation_space.shape or env.observation_space.n
args.action_shape = env.action_space.shape or env.action_space.n
print("Observations shape:", args.state_shape)
print("Actions shape:", args.action_shape)

Observations shape: (52,)
Actions shape: 54


### network

In [56]:
class D5QN(nn.Module):
    """Reference: Human-level control through deep reinforcement learning.

    For advanced usage (how to customize the network), please refer to
    :ref:`build_the_network`.
    """

    def __init__(
        self,
        state_shape: Sequence[int],
        action_shape: Sequence[int],
        device: Union[str, int, torch.device] = "cpu",
        features_only: bool = False,
        output_dim: Optional[int] = None,
    ) -> None:
        super().__init__()
        self.device = device
        self.net = nn.Sequential(
            nn.Linear(state_shape, 512), nn.ReLU(inplace=True),
            nn.Linear(512, action_shape)
        )

    def forward(
        self,
        obs: Union[np.ndarray, torch.Tensor],
        state: Optional[Any] = None,
        info: Dict[str, Any] = {},
    ) -> Tuple[torch.Tensor, Any]:
        r"""Mapping: s -> Q(s, \*)."""
        obs = torch.as_tensor(obs, device=self.device, dtype=torch.float32)
        return self.net(obs), state

### define policy

In [57]:
# define model
net = D5QN(args.state_shape[0], args.action_shape, args.device).to(args.device)
optim = torch.optim.Adam(net.parameters(), lr=args.lr)
# define policy
policy = DQNPolicy(
    net,
    optim,
    args.gamma,
    args.n_step,
    target_update_freq=args.target_update_freq
)

In [58]:
if args.icm_lr_scale > 0:
    feature_net = DQN(
        args.state_shape[0], args.action_shape, args.device, features_only=True
    )
    action_dim = np.prod(args.action_shape)
    feature_dim = feature_net.output_dim
    icm_net = IntrinsicCuriosityModule(
        feature_net.net,
        feature_dim,
        action_dim,
        hidden_sizes=[512],
        device=args.device
    )
    icm_optim = torch.optim.Adam(icm_net.parameters(), lr=args.lr)
    policy = ICMPolicy(
        policy, icm_net, icm_optim, args.icm_lr_scale, args.icm_reward_scale,
        args.icm_forward_loss_weight
    ).to(args.device)

### load policy

In [59]:
# load a previous policy
if args.resume_path:
    policy.load_state_dict(torch.load(args.resume_path, map_location=args.device))
    print("Loaded agent from: ", args.resume_path)

### replay buffer

In [60]:
# replay buffer: `save_last_obs` and `stack_num` can be removed together
# when you have enough RAM
buffer = VectorReplayBuffer(
    args.buffer_size,
    buffer_num=len(train_envs),
    ignore_obs_next=True,
    save_only_last_obs=False,
    stack_num=args.frames_stack
)

### collector

In [61]:
# collector
train_collector = Collector(policy, train_envs, buffer, exploration_noise=True)
test_collector = Collector(policy, test_envs, exploration_noise=True)

### log

In [62]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
args.algo_name = "dqn_icm" if args.icm_lr_scale > 0 else "dqn"
log_name = os.path.join(args.task, args.algo_name, str(args.seed), now)
log_path = os.path.join(args.logdir, log_name)

In [63]:
# logger
if args.logger == "wandb":
    logger = WandbLogger(
        save_interval=1,
        name=log_name.replace(os.path.sep, "__"),
        run_id=args.resume_id,
        config=args,
        project=args.wandb_project,
    )
writer = SummaryWriter(log_path)
writer.add_text("args", str(args))
if args.logger == "tensorboard":
    logger = TensorboardLogger(writer)
else:  # wandb
    logger.load(writer)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hongxi. Use `wandb login --relogin` to force relogin
/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/tmp-data/conda/anaconda3/envs/rl_test/lib/python3.9/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


### train helper funciton

In [64]:
def save_best_fn(policy):
    torch.save(policy.state_dict(), os.path.join(log_path, "policy.pth"))

def stop_fn(mean_rewards):
    return mean_rewards >= 20

def train_fn(epoch, env_step):
    # nature DQN setting, linear decay in the first 1M steps
    if env_step <= 1e6:
        eps = args.eps_train - env_step / 1e6 * \
            (args.eps_train - args.eps_train_final)
    else:
        eps = args.eps_train_final
    policy.set_eps(eps)
    if env_step % 1000 == 0:
        logger.write("train/env_step", env_step, {"train/eps": eps})

def test_fn(epoch, env_step):
    policy.set_eps(args.eps_test)

def save_checkpoint_fn(epoch, env_step, gradient_step):
    # see also: https://pytorch.org/tutorials/beginner/saving_loading_models.html
    ckpt_path = os.path.join(log_path, "checkpoint.pth")
    torch.save({"model": policy.state_dict()}, ckpt_path)
    return ckpt_path

# watch agent's performance
def watch():
    print("Setup test envs ...")
    policy.eval()
    policy.set_eps(args.eps_test)
    test_envs.seed(args.seed)
    if args.save_buffer_name:
        print(f"Generate buffer with size {args.buffer_size}")
        buffer = VectorReplayBuffer(
            args.buffer_size,
            buffer_num=len(test_envs),
            ignore_obs_next=True,
            save_only_last_obs=True,
            stack_num=args.frames_stack
        )
        collector = Collector(policy, test_envs, buffer, exploration_noise=True)
        result = collector.collect(n_step=args.buffer_size)
        print(f"Save buffer into {args.save_buffer_name}")
        # Unfortunately, pickle will cause oom with 1M buffer size
        buffer.save_hdf5(args.save_buffer_name)
    else:
        print("Testing agent ...")
        test_collector.reset()
        result = test_collector.collect(
            n_episode=args.test_num, render=args.render
        )
    rew = result["rews"].mean()
    print(f"Mean reward (over {result['n/ep']} episodes): {rew}")

In [65]:
### train core

In [ ]:
if args.watch:
    watch()
    exit(0)

# test train_collector and start filling replay buffer
train_collector.collect(n_step=args.batch_size * args.training_num)
# trainer
result = offpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    args.epoch,
    args.step_per_epoch,
    args.step_per_collect,
    args.test_num,
    args.batch_size,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_best_fn=save_best_fn,
    logger=logger,
    update_per_step=args.update_per_step,
    test_in_train=False,
    resume_from_log=args.resume_id is not None,
    save_checkpoint_fn=save_checkpoint_fn,
)

pprint.pprint(result)
watch()

Epoch #1: 101it [00:10,  9.71it/s, env_step=100, len=100, loss=92.949, n/ep=1, n/st=100, rew=-842.70]                         


Epoch #1: test_reward: -285.820000 ± 168.290902, best_reward: -285.820000 ± 168.290902 in #1


Epoch #2: 101it [00:08, 12.47it/s, env_step=200, len=100, loss=66.129, n/ep=1, n/st=100, rew=-318.60]                         


Epoch #2: test_reward: -713.090000 ± 285.517202, best_reward: -285.820000 ± 168.290902 in #1


Epoch #3: 101it [00:07, 13.33it/s, env_step=300, len=100, loss=56.188, n/ep=1, n/st=100, rew=-759.70]                         


...

Epoch #704: 101it [00:02, 44.46it/s, env_step=70400, len=100, loss=1.409, n/ep=1, n/st=100, rew=-58.30]                          


Epoch #704: test_reward: -70.700000 ± 34.451909, best_reward: -17.080000 ± 13.172532 in #233


Epoch #705: 101it [00:15,  6.53it/s, env_step=70500, len=100, loss=1.704, n/ep=1, n/st=100, rew=-70.00]                         


Epoch #705: test_reward: -57.700000 ± 30.172206, best_reward: -17.080000 ± 13.172532 in #233


Epoch #706: 101it [00:02, 35.27it/s, env_step=70600, len=100, loss=1.626, n/ep=1, n/st=100, rew=-37.80]                         

libgomp: Thread creation failed: Resource temporarily unavailable
